In [1]:
%matplotlib inline


# Clustering text documents using k-means


This is an example showing how the scikit-learn can be used to cluster
documents by topics using a bag-of-words approach. This example uses
a scipy.sparse matrix to store the features instead of standard numpy arrays.

Two feature extraction methods can be used in this example:

  - TfidfVectorizer uses a in-memory vocabulary (a python dict) to map the most
    frequent words to features indices and hence compute a word occurrence
    frequency (sparse) matrix. The word frequencies are then reweighted using
    the Inverse Document Frequency (IDF) vector collected feature-wise over
    the corpus.

  - HashingVectorizer hashes word occurrences to a fixed dimensional space,
    possibly with collisions. The word count vectors are then normalized to
    each have l2-norm equal to one (projected to the euclidean unit-ball) which
    seems to be important for k-means to work in high dimensional space.

    HashingVectorizer does not provide IDF weighting as this is a stateless
    model (the fit method does nothing). When IDF weighting is needed it can
    be added by pipelining its output to a TfidfTransformer instance.

Two algorithms are demoed: ordinary k-means and its more scalable cousin
minibatch k-means.

Additionally, latent semantic analysis can also be used to reduce dimensionality
and discover latent patterns in the data.

It can be noted that k-means (and minibatch k-means) are very sensitive to
feature scaling and that in this case the IDF weighting helps improve the
quality of the clustering by quite a lot as measured against the "ground truth"
provided by the class label assignments of the 20 newsgroups dataset.

This improvement is not visible in the Silhouette Coefficient which is small
for both as this measure seem to suffer from the phenomenon called
"Concentration of Measure" or "Curse of Dimensionality" for high dimensional
datasets such as text data. Other measures such as V-measure and Adjusted Rand
Index are information theoretic based evaluation scores: as they are only based
on cluster assignments rather than distances, hence not affected by the curse
of dimensionality.

Note: as k-means is optimizing a non-convex objective function, it will likely
end up in a local optimum. Several runs with independent random init might be
necessary to get a good convergence.




## Load in Data

In [2]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Lars Buitinck
# License: BSD 3 clause
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans
from time import time
import numpy as np
import pandas as pd


# #############################################################################
# Load some categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
# Uncomment the following to do the analysis on all the categories
# categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

dataset = fetch_20newsgroups(subset='all', categories=categories,
                             shuffle=True, random_state=42)

print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))
print()


Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
3387 documents
4 categories



## Examine Data and Labels

In [3]:
labels = dataset.target
true_k = np.unique(labels).shape[0]
print("True number of Clusters: ", true_k)

True number of Clusters:  4


## Sample Article

In [4]:
dataset.data[0].split('\n')

['From: healta@saturn.wwc.edu (Tammy R Healy)',
 'Subject: Re: who are we to judge, Bobby?',
 'Lines: 38',
 'Organization: Walla Walla College',
 'Lines: 38',
 '',
 'In article <1993Apr14.213356.22176@ultb.isc.rit.edu> snm6394@ultb.isc.rit.edu (S.N. Mozumder ) writes:',
 '>From: snm6394@ultb.isc.rit.edu (S.N. Mozumder )',
 '>Subject: Re: who are we to judge, Bobby?',
 '>Date: Wed, 14 Apr 1993 21:33:56 GMT',
 '>In article <healta.56.734556346@saturn.wwc.edu> healta@saturn.wwc.edu (TAMMY R HEALY) writes:',
 '>>Bobby,',
 '>>',
 '>>I would like to take the liberty to quote from a Christian writer named ',
 '>>Ellen G. White.  I hope that what she said will help you to edit your ',
 '>>remarks in this group in the future.',
 '>>',
 '>>"Do not set yourself as a standard.  Do not make your opinions, your views ',
 '>>of duty, your interpretations of scripture, a criterion for others and in ',
 '>>your heart condemn them if they do not come up to your ideal."',
 '>>                         Tho

In [5]:
dataset.data[500].split('\n')

['From: tmc@spartan.ac.BrockU.CA (Tim Ciceran)',
 'Subject: Re: Best FTP Viewer please.',
 'Organization: Brock University, St. Catharines Ontario',
 'X-Newsreader: TIN [version 1.1 PL9]',
 'Lines: 19',
 '',
 'SITUNAYA@IBM3090.BHAM.AC.UK wrote:',
 ': ==============================================================================',
 ": Could someone please tell me the Best FTP'able viewer available for MSDOS",
 ': I am running a 486 33mhz with SVGA monitor.',
 ': I need to look at gifs mainly and it would be advantageous if it ran',
 ': under windows...........thanks',
 '',
 'FTP to wuarchive.wustl.edu,',
 'change into mirrors/msdos/graphics',
 'get "grfwk61t.zip"',
 'This is the DOS version of Graphic Workshop.  There is a Windows version which',
 "you could probably find in the mirrors/msdos/windows3 directory but I don't ",
 'know what the file name is. ',
 '',
 '-- ',
 '',
 'TMC',
 '(tmc@spartan.ac.BrockU.ca)',
 '',
 '']

## Converting Text data to Numeric using CountVectorizer

In [6]:
print("Extracting features from the training dataset using a TFidF vectorizer")
t0 = time()
vectorizer = TfidfVectorizer(stop_words='english', 
                             lowercase=True, 
                             ngram_range=(1,2))
X = vectorizer.fit_transform(dataset.data)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

Extracting features from the training dataset using a TFidF vectorizer
done in 3.653120s
n_samples: 3387, n_features: 361840



In [7]:
X.shape

(3387, 361840)

In [8]:
# #############################################################################
# Do the actual clustering

km = KMeans(n_clusters=true_k, max_iter = 1000)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()


Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
done in 161.488s



## Look at the Silhouette Score and provide some other metrics for you to investigate on your own

In [9]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

print()

Homogeneity: 0.478
Completeness: 0.549
V-measure: 0.511
Adjusted Rand-Index: 0.423
Silhouette Coefficient: 0.004



## Lets look at some of the predictions!

In [10]:
pd.DataFrame(list(zip(range(10),km.predict(X)[:10])), columns = ['Observation num.', 'Cluster'])

,Observation num.,Cluster
0,0,0
1,1,3
2,2,3
3,3,3
4,4,3
5,5,3
6,6,3
7,7,1
8,8,1
9,9,0


In [11]:
dataset.data[9].split('\n')

['From: JDB1145@tamvm1.tamu.edu',
 'Subject: Re: A Little Too Satanic',
 'Organization: Texas A&M University',
 'Lines: 21',
 'NNTP-Posting-Host: tamvm1.tamu.edu',
 '',
 'In article <65934@mimsy.umd.edu>',
 'mangoe@cs.umd.edu (Charley Wingate) writes:',
 ' ',
 '>',
 '>Nanci Ann Miller writes:',
 '>',
 ']The "corrupted over and over" theory is pretty weak.  Comparison of the',
 ']current hebrew text with old versions and translations shows that the text',
 ']has in fact changed very little over a space of some two millennia.  This',
 "]shouldn't be all that suprising; people who believe in a text in this manner",
 ']are likely to makes some pains to make good copies.',
 ' ',
 'Tell it to King James, mate.',
 ' ',
 ']C. Wingate        + "The peace of God, it is no peace,',
 ']                  +    but strife closed in the sod.',
 ']mangoe@cs.umd.edu +  Yet, brothers, pray for but one thing:',
 ']tove!mangoe       +    the marv\'lous peace of God."',
 ' ',
 ' ',
 'John Burke, jdb1145@sum

In [12]:
dataset.data[0].split('\n')

['From: healta@saturn.wwc.edu (Tammy R Healy)',
 'Subject: Re: who are we to judge, Bobby?',
 'Lines: 38',
 'Organization: Walla Walla College',
 'Lines: 38',
 '',
 'In article <1993Apr14.213356.22176@ultb.isc.rit.edu> snm6394@ultb.isc.rit.edu (S.N. Mozumder ) writes:',
 '>From: snm6394@ultb.isc.rit.edu (S.N. Mozumder )',
 '>Subject: Re: who are we to judge, Bobby?',
 '>Date: Wed, 14 Apr 1993 21:33:56 GMT',
 '>In article <healta.56.734556346@saturn.wwc.edu> healta@saturn.wwc.edu (TAMMY R HEALY) writes:',
 '>>Bobby,',
 '>>',
 '>>I would like to take the liberty to quote from a Christian writer named ',
 '>>Ellen G. White.  I hope that what she said will help you to edit your ',
 '>>remarks in this group in the future.',
 '>>',
 '>>"Do not set yourself as a standard.  Do not make your opinions, your views ',
 '>>of duty, your interpretations of scripture, a criterion for others and in ',
 '>>your heart condemn them if they do not come up to your ideal."',
 '>>                         Tho

In [13]:
dataset.data[7].split('\n')

['From: 18084TM@msu.edu (Tom)',
 'Subject: Moonbase race',
 'X-Added: Forwarded by Space Digest',
 'Organization: [via International Space University]',
 'Original-Sender: isu@VACATION.VENARI.CS.CMU.EDU',
 'Distribution: sci',
 'Lines: 22',
 '',
 'George William Herbert sez:',
 '',
 '>Hmm.  $1 billion, lesse... I can probably launch 100 tons to LEO at',
 '>$200 million, in five years, which gives about 20 tons to the lunar',
 '>surface one-way.  Say five tons of that is a return vehicle and its',
 '>fuel, a bigger Mercury or something (might get that as low as two',
 ">tons), leaving fifteen tons for a one-man habitat and a year's supplies?",
 '>Gee, with that sort of mass margins I can build the systems off',
 '>the shelf for about another hundred million tops.  That leaves',
 ">about $700 million profit.  I like this idea 8-)  Let's see",
 '>if you guys can push someone to make it happen 8-) 8-)',
 '',
 "I like your optimism, George.  I don't know doots about raising that kind",
 'of

In [14]:
dataset.data[8].split('\n')

['From: baalke@kelvin.jpl.nasa.gov (Ron Baalke)',
 "Subject: JPL's VLBI Project Meets with International Space Agencies",
 'Organization: Jet Propulsion Laboratory',
 'Lines: 112',
 'Distribution: world',
 'NNTP-Posting-Host: kelvin.jpl.nasa.gov',
 'Keywords: VLBI, JPL',
 'News-Software: VAX/VMS VNEWS 1.41    ',
 '',
 'From the "JPL Universe"',
 'April 23, 1993',
 '',
 'VLBI project meets with international space agencies',
 '',
 'By Ed McNevin',
 "     Members of JPL's Space Very Long Baseline Interferometry",
 '(VLBI) project team recently concluded a week-long series of',
 'meetings with officials from Russia and Japan.',
 '     The meetings were part of "Space VLBI Week" held at JPL in',
 'early March and were intended to maintain cooperation between',
 'international space agencies participating in the development of',
 'the U.S. Space VLBI Project, a recently approved JPL flight',
 'project set for launch in 1995.',
 '     U.S. Space VLBI will utilize two Earth-orbiting spacecraf